In [105]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

In [111]:
df_train = pd.read_csv('train_final.csv')
df_test = pd.read_csv('test_final.csv')

In [107]:
df_test = df_test.drop('ID',axis = 1)

In [16]:
# for col in ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'sex', 'race', 'native.country']:
#     one_hot_train = pd.get_dummies(df_train[col], prefix=col)
#     one_hot_test = pd.get_dummies(df_test[col], prefix=col)

#     df_train = df_train.drop(col,axis = 1)
#     df_train = pd.concat([df_train,one_hot_train],axis=1)

#     df_test = df_test.drop(col,axis = 1)
#     df_test = pd.concat([df_test,one_hot_test],axis=1)


In [114]:
label_encoder = preprocessing.LabelEncoder()
for col in ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'sex', 'race', 'native.country']:
    df_test[col] = label_encoder.fit_transform(df_test[col]) 
    df_train[col]= label_encoder.transform(df_train[col]) 
    

In [18]:
missing_col = None
for col in df_test.columns:
    if col not in df_train.columns:
        df_train[col] = 0
        print(col)

native.country_Holand-Netherlands


In [117]:
X = df_train.loc[:, df_train.columns != 'income>50K']
y = df_train['income>50K']

X_test_final = df_test.loc[:, df_test.columns != 'income>50K']

In [136]:
estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

parameters = {
    'max_depth': [3,4,5],
    'n_estimators': [200],
    'learning_rate': [0.3, 0.2],
    'gamma': [0, 1, 5],
    "min_child_weight":[1]
}

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'accuracy',
    cv = 5,
    verbose=True
)

In [137]:
grid_search.fit(X,y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_ca...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                  

In [138]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid_search.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_search.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_search.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=200,
              n_jobs=4, nthread=4, num_parallel_tree=1, predictor='auto',
              random_state=42, reg_alpha=0, ...)

 The best score across ALL searched params:
 0.8748799999999999

 The best parameters across ALL searched params:
 {'gamma': 0, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200}


In [139]:
best_params = {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 360}
best_params2 = {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 5, 'n_estimators': 300}
best_params_labelencoded =  {'gamma': 0, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200}

In [140]:
model = XGBClassifier(**best_params_labelencoded)
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=200,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [144]:
y_pred = model.predict(X_test_final)

In [145]:
import csv
ctr = 1
data = [['ID', 'Prediction']]
for pred in y_pred:
    row = [ctr, pred]
    data.append(row)
    ctr += 1
# open the file in the write mode
with open('results.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data)

In [132]:
w = np.asarray([1,1,0])
x=np.asarray([0,1,1])
w@x

1

In [133]:
1 / np.linalg.norm(w)

0.7071067811865475